### Bayesian network inference library

We will be using the pomegranate library for Bayes Net inference

  * Installation instructions https://pomegranate.readthedocs.io/en/latest/install.html
  * Tutorial / documentation https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html
  
In the tutorial / documentation, ignore the parts about "initializing a Bayesian network based completely on data" and the sections on "Probability" "Prediction" and "Fitting" -- see the example below on how to determine the probability distribution on a node in the graph based on evidence.

Just to make sure things are working, first load in the Monty Hall code from the tutorial and try 
  * 

In [2]:
from pomegranate import *

guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
monty = ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest, prize])

s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()

In [3]:
##  Suppose the guest chooses A, and Monty chooses B.
##  Monty gives the guest to switch from A to C.  Should she?
model.predict_proba({"guest": 'A', "monty": 'B'})

array(['A',
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "A" :0.3333333333333334,
            "B" :0.0,
            "C" :0.6666666666666664
        }
    ],
    "frozen" :false
},
       'B'], dtype=object)

In [2]:
from pomegranate import *

#### Here's an example of a noisy sensor ####

* The variable **water** says whether or not there is water in my basement.  This variable takes values **{none, some, lots}**
* I have a water detector **waterDetector** that is either **on** or **off**
  * It is supposed to be ON if and only if W is either some or lots
  * However, it sometimes fails to alert (is **off** when **water** is either **some** or **lots**)
  * It also sometimes false alarms (is **on** when **water** is **none**)

This is what I discovered by observing the basement over time
* On any given day, the probability of **water** is **(.98, .015, .05)** for values **(none, some, lots)**
* The likelihood of a false alarm **P(waterDetector = on | water = none) = 0.01**
* The likelihood of the sensor missing water depends on the water level: **P(waterDetector = off | water = some) = .10**;   **P(waterDetector = off | water = lots) = .005**


In [3]:
# The distributions
wdist = DiscreteDistribution({"none": 0.98, "some": 0.015, "lots": 0.005})

wddist = ConditionalProbabilityTable([["none", "on", 0.01],  ["none", "off", 0.99],
                                      ["some", "on", 0.90],  ["some", "off", 0.10],
                                      ["lots", "on", 0.995], ["lots", "off", 0.005]],
                                     [wdist])

# The nodes
water = Node(wdist, name="water")
waterDetector = Node(wddist, name="waterDetector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(water, waterDetector)
model.add_edge(water, waterDetector)
model.bake()
                                     

With no further information, what is the likelihood that (a) there is some or lots of water in my basement, and (b) what is the likelihood that my water detector is displaying ON

In [5]:
model.marginal()

array([{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "none" :0.9799999999999991,
            "some" :0.015000000000000421,
            "lots" :0.005000000000000436
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "on" :0.028275000000000623,
            "off" :0.9717249999999994
        }
    ],
    "frozen" :false
}], dtype=object)

In [4]:
# Compute probabilities on the basis of no additional evidence.  Its output is a list of 
# distributions over node values, in the order they were added -- in our case, water is at [0] and waterDetector is at [1]
model.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "none" :0.9799999999999991,
            "some" :0.015000000000000421,
            "lots" :0.005000000000000436
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "on" :0.028275000000000623,
            "off" :0.9717249999999994
        }
    ],
    "frozen" :false
}], dtype=object)

In [20]:
# This is the distribution over values of water -- not surprising, it is the same as the priors 
#  (subject to rounding error)
model.predict_proba({})[0].parameters

[{'none': 0.9799999999999991,
  'some': 0.015000000000000421,
  'lots': 0.005000000000000436}]

In [24]:
# This is the distribution over values of waterDetector
model.predict_proba({})[1].parameters

[{'on': 0.028275000000000623, 'off': 0.9717249999999994}]

Suppose I learn that the water detector is **on**.  How does that affect my beliefs over the basement water level

In [21]:
model.predict_proba({"waterDetector": "on"})[0].parameters

[{'none': 0.34659593280282863,
  'some': 0.4774535809018497,
  'lots': 0.17595048629532176}]

In [ ]:
Suppose instead I go to the basement and observe that there is no water in the basement.  
How does that affect my belief as to whether or not the water detector is on?

In [23]:
model.predict_proba({"water": "none"})[1].parameters

[{'on': 0.010000000000000222, 'off': 0.9899999999999998}]

####  Question 1 ####
My beliefs about water level change with the season.   There are two seasons, the dry season and the wet season.
The season affects my prior belief in the water level, not the behavior of the sensor.   If the season is 
**wet**  my prior distribution on **water** is
{"none": 0.80, "some": 0.15, "lots": 0.05})
and if the season is **dry** the distribution is 
{"none": 0.95, "some": 0.035, "lots": 0.015})

Adjust the model accordingly.  Suppose it's the dry season, but my water detector is saying **on** -- what do I believe about water in the basement?


In [3]:
# This is my season node
sdist = DiscreteDistribution({"dry": 0.50, "wet": 0.50})
season = Node(sdist, name="season")

# This is my water node
wdist = ConditionalProbabilityTable([["dry", "none", 0.95],  ["dry", "some", 0.035], ["dry", "lots", 0.015],
                                     ["wet", "none", 0.80],  ["wet", "some", 0.15],  ["wet", "lots", 0.05]], 
                                    [sdist])
water = Node(wdist, name="water")

# My water detector node
wddist = ConditionalProbabilityTable([["none", "on", 0.01],  ["none", "off", 0.99],
                                      ["some", "on", 0.90],  ["some", "off", 0.10],
                                      ["lots", "on", 0.995], ["lots", "off", 0.005]],
                                     [wdist])
waterDetector = Node(wddist, name="waterDetector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(season, water, waterDetector)
model.add_edge(season, water)
model.add_edge(water, waterDetector)
model.bake()

In [4]:
model.predict_proba({"season": "dry", "waterDetector": "on"})

array(['dry',
       {
    "class" :"Distribution",
    "dtype" :"str",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "some" :0.5632543585158643,
            "none" :0.16987036209209097,
            "lots" :0.2668752793920448
        }
    ],
    "frozen" :false
},
       'on'], dtype=object)

#### Question 2 ####

Code the example from the lectures, about burglaries, alarms, and phone calls
* Variables are B (Burglary), E (Earthquake), A (Alarm), J (John calls), M (Mary calls)
  * All are true/false
* Assumptions we make as domain experts
  * Burglaries do not cause earthquakes, and vice versa
  * The alarm's behavior depends only on B and E and uncorrelated error behavior (for example, if there is no Burglary or Earthquake, then the likelihood the alarm sounds anyway does not depend on any other factor)
  * John and Mary have their own parameters, but those depend only on A, and otherwise they act independently
* Parameters we have gathered
  * Prob(B) = .001
  * Prob(E) = .002
  * Prob(A | B,E) = (.95, .94, .29, .001)    ((+b, +e), (+b, -e), (-b, +e), (-b, -e))
  * Prob(J | A) = (.9, .45)
  * Prob(M | A) = (.7, .01).


Code this as a Bayes network, and answer these two questions:

1.  My phone is off so I don't know whether or not I got a call.  What is the likelihood there was a burglary?  What is the likelihood my alarm went off?

2.  I just got a call from Mary but not John.  How does that affect my belief that there was a burglary?

3.  I heard a reliable news report that there was an earthquake.  What is the chance I will will be getting a call from John soon?


In [19]:
from pomegranate import *

bdist = DiscreteDistribution({1: .001, 0: (1 - .001)})
b = Node(bdist, name="burglary")

edist = DiscreteDistribution({1: .002, 0: (1 - .002)})
e = Node(edist, name="earthquake")


adist = ConditionalProbabilityTable([[1, 1,  1, .95],   [1, 1, 0, (1-.95)],
                                     [1, 0,  1, .94],   [1, 0, 0, (1-.94)],
                                     [0, 1,  1, .29],   [0, 1, 0, (1-.29)],
                                     [0, 0,  1, .001],  [0, 0, 0, (1-.001)]],
                                   [bdist, edist])
a = Node(adist, name="alarm")

jdist = ConditionalProbabilityTable([[1, 1, .9],   [1, 0, (1-.9)],
                                     [0, 1, .05],  [0, 0, (1-.05)]],
                                    [adist])
j = Node(jdist, name="john")

mdist = ConditionalProbabilityTable([[1, 1, .7],  [1, 0, (1-.7)],
                                     [0, 1, .01],  [0, 0, (1-.01)]],
                                    [adist])
m = Node(mdist, name="mary")

# The Network
model = BayesianNetwork("Alarm")
model.add_states(b,e, a, j, m)
model.add_edge(b,a)
model.add_edge(e,a)
model.add_edge(a,j)
model.add_edge(a,m)
model.bake()


In [27]:
model.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "1" :0.001000000000000444,
            "0" :0.9989999999999996
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "1" :0.002000000000000442,
            "0" :0.9979999999999996
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.997483557999999,
            "1" :0.002516442000000935
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtype" :"int",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "0" :0.9478610242999992,
            "1" :0.0521389757000008
        }
    ],
    "frozen" :false
},
       {
    "class" :"Distribution",
    "dtyp